In [ ]:
# !open .

In [1]:
import pandas as pd
import requests
import bs4
import math
import time
import NeoQuant as nq
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
pd.set_option('display.max_colwidth', -1)
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

## get all company list

In [3]:
companies = nq.get_all_company_list()

## get fnguide snapshot

In [9]:
company_name = '포스코'

In [10]:
company_code = nq.get_company_code(company_name, companies)
print('company name : ' + company_name)
print('company code : ' + company_code)

if not company_code.startswith('A'):
    company_code = 'A' + company_code

snapshot_url = 'http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=701&gicode=' + company_code
snapshot_page = requests.get(snapshot_url)
snapshot_tables = pd.read_html(snapshot_page.text)

company name : 포스코
company code : 005490


## bond spread

In [11]:
# https://www.kisrating.co.kr/ratingsStatistics/statics_spread.do
expected_ratio = 7.91

## get stock count

In [13]:
import math

def get_total_stock_count(snapshot_tables):
    stock_count = 0

    info = snapshot_tables[0]
    info = info.set_index(info.columns[0])
    stock_count_info = info.loc['발행주식수(보통주/ 우선주)'][1]
    stock_counts = stock_count_info.split('/')
    for count in stock_counts:
#         print(count)
        stock_count = stock_count + int(count.replace(',',''))

    return stock_count

def get_self_stock_count(snapshot_tables):
    self_stock_count = 0
    info = snapshot_tables[4]
    info = info.set_index(info.columns[0])
    count = info.loc[['자기주식\xa0(자사주+자사주신탁)']]['보통주'][0]
#     print(count)
    if not math.isnan(count):
        self_stock_count = int(count)
    return self_stock_count

stock_count = get_total_stock_count(snapshot_tables) - get_self_stock_count(snapshot_tables)
print('stock count : ' + str(stock_count))

stock count : 76015472


In [15]:
snapshot_tables[10]

IFRS(연결)     Annual                                  Net Quarter  \
      IFRS(연결)    2017/12    2018/12    2019/12 2020/12(P)     2020/03   
0   매출액         606551.00  649778.00  643668.00  577928.00  145458.00    
1   영업이익        46218.00   55426.00   38689.00   24030.00   7053.00      
2   영업이익(발표기준)  46218.00   55426.00   38689.00   24030.00   7053.00      
3   당기순이익       29735.00   18921.00   19826.00   17882.00   4347.00      
4   지배주주순이익     27901.00   16906.00   18351.00   16021.00   3954.00      
5   비지배주주순이익    1834.00    2015.00    1476.00   NaN         393.00       
6   자산총계        790250.00  782483.00  790587.00 NaN         822607.00    
7   부채총계        315610.00  314887.00  312640.00 NaN         348463.00    
8   자본총계        474640.00  467596.00  477947.00 NaN         474144.00    
9   지배주주지분      437329.00  433713.00  444719.00 NaN         442346.00    
10  비지배주주지분     37311.00   33883.00   33228.00  NaN         31797.00     
11  자본금         4824.00    4824.00    4824.00   NaN         4824.00      
12  부채비율        66.49      67.34      65.41     NaN         73.49        
13  유보율         9283.42    9208.39    9431.48   NaN         9382.31      
14  영업이익률       7.62       8.53       6.01      NaN         4.85         
15  지배주주순이익률    4.60       2.60       2.85       3.09       2.72         
16  ROA         3.75       2.41       2.52      NaN         2.16         
17  ROE         6.48       3.88       4.18      NaN         3.57         
18  EPS(원)      32001.00   19391.00   21048.00   18375.00   4535.00      
19  BPS(원)      519183.00  515032.00  527375.00 NaN         524654.00    
20  DPS(원)      8000.00    10000.00   10000.00  NaN         1500.00      
21  PER         10.39      12.53      11.24     NaN        NaN           
22  PBR         0.64       0.47       0.45      NaN         0.31         
23  발행주식수       87187.00   87187.00   87187.00  NaN         87187.00     
24  배당수익률       2.41       4.12       4.23      NaN         0.93         

                                     
      2020/06    2020/09 2020/12(P)  
0   137216.00  142612.00  152642.00  
1   1677.00    6667.00    8633.00    
2   1677.00    6667.00    8633.00    
3   1049.00    5140.00    7346.00    
4   403.00     4655.00    7010.00    
5   646.00     486.00    NaN         
6   821118.00  820912.00 NaN         
7   345187.00  343008.00 NaN         
8   475931.00  477904.00 NaN         
9   443309.00  444368.00 NaN         
10  32622.00   33536.00  NaN         
11  4824.00    4824.00   NaN         
12  72.53      71.77     NaN         
13  9415.70    9492.55   NaN         
14  1.22       4.67      NaN         
15  0.29       3.26       4.81       
16  0.51       2.50      NaN         
17  0.36       4.19      NaN         
18  462.00     5339.00    8040.00    
19  526502.00  530754.00 NaN         
20  500.00     1500.00   NaN         
21 NaN        NaN        NaN         
22  0.33       0.37      NaN         
23  87187.00   87187.00  NaN         
24  0.29       0.77      NaN

In [23]:
big_col_name = 'Annual'
roe_index_name = 'ROE'
asset_index_name = '지배주주지분'

info = snapshot_tables[10]
info = info.set_index(info.columns[0])

In [35]:
annual_index = info['Annual'].columns[:3]

In [38]:
','.join(annual_index)

'2017/12,2018/12,2019/12'

## get asset, roe

In [22]:
import math

def get_standard_col_name(snapshot_tables):
    big_col_name = 'Annual'
    roe_index_name = 'ROE'
    asset_index_name = '지배주주지분'
    
    info = snapshot_tables[10]
    info = info.set_index(info.columns[0])

    stadard_col_name = ''
    roes = info.loc[roe_index_name][big_col_name]
    assets = info.loc[asset_index_name][big_col_name]
    for index in reversed(assets.index):
        if not math.isnan(assets[index]) and not math.isnan(roes[index]):
            stadard_col_name = index
            break
    return stadard_col_name

def get_roe(snapshot_tables, stadard_col_name):
    big_col_name = 'Annual'
    roe_index_name = 'ROE'
    
    info = snapshot_tables[10]
    info = info.set_index(info.columns[0])

    roe = info.loc[roe_index_name][big_col_name, stadard_col_name]
    return roe

def get_net_profit(snapshot_tables, stadard_col_name):
    big_col_name = 'Annual'
    profit_index_name = '지배주주지분'
    
    info = snapshot_tables[10]
    info = info.set_index(info.columns[0])

    profit = info.loc[profit_index_name][big_col_name, stadard_col_name]
    profit = profit * 100000000
    return profit

def get_op_profit(snapshot_tables, stadard_col_name):
    big_col_name = 'Annual'
    profit_index_name = '영업이익'
    
    info = snapshot_tables[10]
    info = info.set_index(info.columns[0])

    profit = info.loc[profit_index_name][big_col_name, stadard_col_name]
    profit = profit * 100000000
    return profit

def get_asset(snapshot_tables, stadard_col_name):
    big_col_name = 'Annual'
    asset_index_name = '자산총계'
    
    info = snapshot_tables[10]
    info = info.set_index(info.columns[0])

    asset = info.loc[asset_index_name][big_col_name, stadard_col_name]
    asset = asset * 100000000
    return asset


stadard_col_name = get_standard_col_name(snapshot_tables)
roe =  get_roe(snapshot_tables, stadard_col_name)
net_profit = get_net_profit(snapshot_tables, stadard_col_name)
op_profit = get_op_profit(snapshot_tables, stadard_col_name)
asset = get_asset(snapshot_tables, stadard_col_name)
roa = round(op_profit / asset * 100, 2)
# round(float(rate * 100), 2)
print('standard date : ' + stadard_col_name)
print('net profit : ' + str(net_profit))
print('op profit : ' + str(op_profit))
print('asset : ' + str(asset))
print('roe : ' + str(roe))
print('roa : ' + str(roa))

standard date : 2019/12
net profit : 44471900000000.0
op profit : 3868900000000.0
asset : 79058700000000.0
roe : 4.18
roa : 4.89


## set roe manually

In [ ]:
# roe = 20

## get adequate price

In [ ]:
def get_adequate_price(asset, roe, expected_ratio, stock_count, persist_factor = 1):
    excess_profit = (roe - expected_ratio) * asset / 100
#     print(excess_profit)
    accumulate_profit = (persist_factor * excess_profit) / (1 + expected_ratio / 100 - persist_factor)
    adequate_stock_price = asset + accumulate_profit
#     print(adequate_stock_price)
    price = (asset + accumulate_profit) / stock_count
    price = int(round(price))
    return price

price_high = get_adequate_price(asset, roe, expected_ratio, stock_count, 1)
price_middel = get_adequate_price(asset, roe, expected_ratio, stock_count, 0.9)
price_low = get_adequate_price(asset, roe, expected_ratio, stock_count, 0.8)

print('buy : below ' + str(price_low))
print('sell 1/3 : ' + str(price_middel))
print('sell 1/3 : ' + str(price_high))

In [ ]:
price_df = request_price_list(firm_code, 'day', 1000)

In [ ]:
price_df['price_low'] = [price_low] * len(price_df)
price_df['price_middle'] = [price_middel] * len(price_df)
price_df['price_high'] = [price_high] * len(price_df)

In [ ]:
show_price_chart(firm_name, firm_code, price_df)

In [ ]:
def get_ex_profits(asset, expected_ratio, roe, persist_factor, iteration = 10):
    next_asset = asset
    ex_profits = []
    ex_profit_ratio = roe - expected_ratio
    next_roe = roe
    for _ in range(iteration):
        ex_profit_ratio = ex_profit_ratio * persist_factor
        next_roe = expected_ratio + ex_profit_ratio
        profit = next_asset * (next_roe / 100)
        ex_profit = next_asset * ex_profit_ratio / 100
        ex_profits.append(ex_profit)
        next_asset = next_asset + profit
    return ex_profits
def get_npv_profit(ex_profits, expected_ratio):
    npv_value = 0
    for num, ex_profit in enumerate(ex_profits):
        ex_profit = ex_profit *  1 / (1 + expected_ratio / 100) ** (num + 1)
#         print(ex_profit)
        npv_value = npv_value + ex_profit

    return npv_value

def get_sum_of_profit(asset, expected_ratio, roe, persist_factor, iteration = 10):
    ex_profits = get_ex_profits(asset, expected_ratio, roe, persist_factor, iteration)
    sum_of_profit = get_npv_profit(ex_profits, expected_ratio)
    return sum_of_profit

def get_more_adequate_price(asset, roe, expected_ratio, stock_count, persist_factor = 1, iteration = 10):
    accumulate_profit = get_sum_of_profit(asset, expected_ratio, roe, persist_factor, iteration)
    adequate_stock_price = asset + accumulate_profit
    price = (asset + accumulate_profit) / stock_count
    price = int(round(price))
    return price

In [ ]:
get_sum_of_profit(asset, expected_ratio, roe, 1, 10)

In [ ]:
price_high = get_more_adequate_price(asset, roe, expected_ratio, stock_count, 1)
price_middel = get_more_adequate_price(asset, roe, expected_ratio, stock_count, 0.9)
price_low = get_more_adequate_price(asset, roe, expected_ratio, stock_count, 0.8)

print('buy : below ' + str(price_low))
print('sell 1/3 : ' + str(price_middel))
print('sell 1/3 : ' + str(price_high))

In [ ]:
price_df = request_price_list(firm_code, 'day', 1000)

In [ ]:
price_df['price_low'] = [price_low] * len(price_df)
price_df['price_middle'] = [price_middel] * len(price_df)
price_df['price_high'] = [price_high] * len(price_df)

In [ ]:
show_price_chart(firm_name, firm_code, price_df)